pip install awscli sagemaker --user

In [2]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(framework_version="2.2.2", model_data='s3://model-deploy/cpm-lm-tf2-fp16.tar.gz', role='arn:aws-cn:iam::907067680148:role/service-role/AmazonSageMaker-ExecutionRole-20210422T153180')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [3]:
predictor.predict({"inp":[1,2,3,4], "length":10})

{'predictions': [[1,
   2,
   3,
   4,
   48,
   11014,
   8,
   3,
   3227,
   48,
   8862,
   8,
   3,
   9338,
   48]]}

In [4]:
import tensorflow as tf
import json
from gpt2_tokenizer import GPT2Tokenizer

tokenizer = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')


length = 400
sentence = "罗俊是中山大学的校长"
inputs = tf.constant([tokenizer.encode(sentence)], dtype=tf.int64)
length = tf.constant(length, dtype=tf.int64)
inputs = inputs.numpy().tolist()
inputs = inputs[0]
print(inputs)
data = json.dumps({"inp":[837, 259, 497, 46, 63, 8, 9, 18892, 328, 169, 1252], "length":50})
ret = predictor.predict({"inp":[1659, 4135, 35, 15027, 11, 4379], "length":400})

ret = ret['predictions']

print([tokenizer.decode(s).replace(' ', '') for s in ret])

ModuleNotFoundError: No module named 'gpt2_tokenizer'

In [8]:
from sagemaker.predictor import Predictor
predictor = Predictor(endpoint_name="tensorflow-inference-2021-04-29-16-42-15-011")

In [13]:
import json
data = {"inp":[1659, 4135, 35, 15027, 11, 4379], "length":400}
data = json.dumps(data)

predictor.predict(data, initial_args={'ContentType': 'application/json'})

b'{\n    "predictions": [[1659, 4135, 35, 15027, 11, 4379, 8, 9, 28, 79, 1000, 11, 13251, 26, 677, 35, 1330, 5671, 37, 19884, 11, 729, 18894, 5001, 8, 12, 8, 10, 19, 78, 8, 418, 61, 107, 16, 951, 68, 39, 249, 11, 119, 928, 8, 2842, 39, 249, 20, 8, 10, 331, 62, 8, 12, 8, 10, 331, 8, 14173, 5755, 5044, 8, 12, 8, 10, 6746, 1012, 3078, 8, 9, 129, 269, 780, 194, 4025, 372, 49, 716, 8, 375, 11, 1508, 11, 82, 358, 2207, 89, 8, 12, 8, 10, 630, 507, 928, 393, 16, 8, 9, 1012, 145, 19, 737, 31, 5601, 27, 1892, 321, 20, 8, 10, 834, 1134, 8, 9, 27, 5601, 16, 5601, 16, 45, 8, 10, 16, 8, 9, 16, 77, 5601, 149, 45, 8, 10, 28, 8, 9, 16, 43, 11, 82, 13116, 81, 345, 15, 78, 8, 12, 8, 10, 16, 8, 9, 16, 223, 106, 5601, 27, 8, 12, 8, 10, 28, 8, 9, 27, 80, 226, 43, 45, 8, 10, 16, 8, 9, 58, 8, 10, 16, 8, 9, 226, 43, 45, 8, 10, 28, 8, 9, 16, 540, 226, 8, 9, 57, 20, 8, 10, 16, 8, 9, 58, 8, 10, 28, 8, 9, 82, 237, 133, 8, 9, 82, 81, 563, 3846, 8, 12, 8, 10, 25, 594, 321, 16, 223, 80, 1382, 3471, 8, 12, 8, 10, 19, 

In [11]:
predictor.accept

('*/*',)